### Load Data
- `events.csv`: Detailed match events (e.g., goals, fouls, shots).
- `ginf.csv`: General match details (e.g., teams, league, date).

Data is loaded into DataFrames, and row counts are printed for verification.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#CONSTS
READ_FOLDER_PATH = '../data/raw/'
WRITE_FOLDER_PATH = '../data/processed/'

events_df = pd.read_csv(READ_FOLDER_PATH+'events.csv')
ginf_df = pd.read_csv(READ_FOLDER_PATH+'ginf.csv')

print(events_df.shape[0])
print(ginf_df.shape[0])

941009
10112


### Merge and Filter Datasets
Merges `events_df` and `ginf_df` on `id_odsp` (unique match identifier).
Retains key fields such as date, teams, goals, event types, and betting odds.  
Sorts data by match date, ID, and event time to ensure proper sequence.  
Prints dataset structure and size to confirm the merge.

In [2]:
# Merge the datasets on 'id_odsp' 
merged_table = pd.merge(events_df, ginf_df, on='id_odsp', how='inner') 
  
#select fields to keep: 
merged_table = merged_table[['id_odsp', 'date', 'league','season', 'country','ht','at', 'time','text', 'event_type','event_type2', 'shot_outcome','is_goal','side', 'event_team', 'opponent', 'fthg','ftag', 'odd_h', 'odd_d','odd_a']]

#sort data by date and then by id_odsp and time 
merged_table = merged_table.sort_values(by=['date', 'id_odsp', 'time'])

print(merged_table.shape[0]) 
print(merged_table.head()) 

941009
     id_odsp        date league  season  country                 ht  \
0  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
1  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
2  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
3  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
4  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   

           at  time                                               text  \
0  Hamburg SV     2  Attempt missed. Mladen Petric (Hamburg) left f...   
1  Hamburg SV     4  Corner,  Borussia Dortmund. Conceded by Dennis...   
2  Hamburg SV     4  Corner,  Borussia Dortmund. Conceded by Heiko ...   
3  Hamburg SV     7           Foul by Sven Bender (Borussia Dortmund).   
4  Hamburg SV     7  Gokhan Tore (Hamburg) wins a free kick in the ...   

   event_type  ...  shot_outcome  is_goal  side         event_team  \
0           1  ...           2.0        0     2    

### Calculate Game Statistics
Defines a function, `calculate_game_stats`, to compute cumulative match-level statistics:
- Tracks goals, shots (on/off target), fouls, yellow cards, red cards, and corners for both teams.
- Determines game state (winning, losing, or drawing) in real-time based on the score.
- Iterates through match events, updating stats as the game progresses.
Applies the function to group events by match (`id_odsp`) and appends new columns for the calculated stats.

In [3]:
# Initialise a function to calculate and append the tallies for each game
def calculate_game_stats(df): 
    home_score, away_score = 0, 0
    home_shots, away_shots = 0, 0
    home_shots_on_target, away_shots_on_target = 0, 0
    home_shots_off_target, away_shots_off_target = 0, 0
    home_yellow_cards, away_yellow_cards = 0,0
    home_sending_off, away_sending_off = 0,0
    home_fouls, away_fouls = 0,0
    home_corners, away_corners = 0,0

    stats = {
        'score': [],
        'game_state':[],
        'home_shots': [], 'away_shots':[],
        'home_shots_on_target': [],'away_shots_on_target':[],
        'home_shots_off_target': [],'away_shots_off_target': [],
        'home_yellow_cards': [], 'away_yellow_cards': [],
        'home_sending_off': [], 'away_sending_off':[],
        'home_fouls': [], 'away_fouls': [],
        'home_corners': [], 'away_corners': []
    }

    for _, row in df.iterrows():
        # Tally goals
        if row['is_goal'] == 1:
            if row['side'] == 1:
                home_score += 1
            elif row['side'] == 2:
                away_score += 1
 
        if row['side'] == 1: #home team
            if home_score > away_score:
                stats['game_state'].append('winning')
            elif home_score < away_score:
                stats['game_state'].append('losing')
            else:
                stats['game_state'].append('drawing')
        elif row['side'] == 2:  #away team
            if away_score > home_score:
                stats['game_state'].append('winning')
            elif away_score < home_score:
                stats['game_state'].append('losing')
            else:
                stats['game_state'].append('drawing')
        else:
            stats['game_state'].append('drawing')

        #Tally shots
        if row['event_type'] == 1:  #attempt/shot
            if row['side'] == 1:
                home_shots += 1
                if row['shot_outcome'] == 1:  #On target
                    home_shots_on_target += 1
                elif row['shot_outcome'] in [2,3,4]:  #Off target
                    home_shots_off_target += 1
            elif row['side'] == 2:
                away_shots += 1
                if row['shot_outcome'] == 1:
                    away_shots_on_target += 1
                elif row['shot_outcome'] in [2,3,4]:
                    away_shots_off_target += 1

        #Tally yellow cards
        if row['event_type'] == 4: #yellow card
            if row['side'] == 1:
                home_yellow_cards += 1
            elif row['side'] == 2:
                away_yellow_cards += 1
 
        #Tally red cards
        if row['event_type2'] == 14:  #Red card
            if row['side'] == 1:
                home_sending_off += 1
            elif row['side'] == 2:
                away_sending_off += 1

        #Tally fouls
        if row['event_type'] == 3:  #foul
            if row['side'] == 1:
                home_fouls += 1
            elif row['side'] == 2:
                away_fouls += 1

        #Tally corners
        if row['event_type'] == 2:  #corner
            if row['side'] == 1:
                home_corners += 1
            elif row['side'] == 2:
                away_corners += 1

        # Append the current tallies to the lists
        stats['score'].append(f"{home_score}-{away_score}")
        stats['home_shots'].append(home_shots)
        stats['away_shots'].append(away_shots) 
        stats['home_shots_on_target'].append(home_shots_on_target)
        stats['away_shots_on_target'].append(away_shots_on_target) 
        stats['home_shots_off_target'].append(home_shots_off_target)
        stats['away_shots_off_target'].append(away_shots_off_target)

        stats['home_yellow_cards'].append(home_yellow_cards) 
        stats['away_yellow_cards'].append(away_yellow_cards)
        stats['home_sending_off'].append(home_sending_off)
        stats['away_sending_off'].append(away_sending_off) 
        stats['home_fouls'].append(home_fouls)
        stats['away_fouls'].append(away_fouls)
        stats['home_corners'].append(home_corners)
        stats['away_corners'].append(away_corners) 


    #Add to df
    for key, values in stats.items():
        df[key] = values

    return df
 
data = merged_table.groupby('id_odsp', group_keys=False).apply(calculate_game_stats)
#Sort by date, id_odsp, and time 
data=data.sort_values(by=['date', 'id_odsp', 'time']).reset_index(drop=True)

print(data.head()) 
data 

/var/folders/hy/8_c5d96s74729952xdwzp2sm0000gn/T/ipykernel_40975/1837944200.py:117: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = merged_table.groupby('id_odsp', group_keys=False).apply(calculate_game_stats)


     id_odsp        date league  season  country                 ht  \
0  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
1  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
2  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
3  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   
4  UFot0hit/  2011-08-05     D1    2012  germany  Borussia Dortmund   

           at  time                                               text  \
0  Hamburg SV     2  Attempt missed. Mladen Petric (Hamburg) left f...   
1  Hamburg SV     4  Corner,  Borussia Dortmund. Conceded by Dennis...   
2  Hamburg SV     4  Corner,  Borussia Dortmund. Conceded by Heiko ...   
3  Hamburg SV     7           Foul by Sven Bender (Borussia Dortmund).   
4  Hamburg SV     7  Gokhan Tore (Hamburg) wins a free kick in the ...   

   event_type  ...  home_shots_off_target  away_shots_off_target  \
0           1  ...                      0                   

,id_odsp,date,league,season,country,ht,at,time,text,event_type,...,home_shots_off_target,away_shots_off_target,home_yellow_cards,away_yellow_cards,home_sending_off,away_sending_off,home_fouls,away_fouls,home_corners,away_corners
0,UFot0hit/,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,...,0,1,0,0,0,0,0,0,0,0
1,UFot0hit/,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,...,0,1,0,0,0,0,0,0,1,0
2,UFot0hit/,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,...,0,1,0,0,0,0,0,0,2,0
3,UFot0hit/,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,7,Foul by Sven Bender (Borussia Dortmund).,3,...,0,1,0,0,0,0,1,0,2,0
4,UFot0hit/,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,...,0,1,0,0,0,0,1,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,...,7,9,1,3,0,0,13,18,5,3
941005,z5L2OT5E/,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,...,7,9,1,3,0,0,13,18,5,4
941006,z5L2OT5E/,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,...,7,10,1,3,0,0,13,18,5,4
941007,z5L2OT5E/,2017-01-22,I1,2017,italy,Atalanta,Sampdoria,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,...,7,10,1,3,0,0,13,18,5,4


### Save Processed Data
Saves the processed dataset with all statistics to `game_statistics.csv`.  
A snippet of the first 500 rows is saved as `game_statistics_snippet.csv`.  

In [4]:
print(data.shape[0])

data.to_csv(WRITE_FOLDER_PATH+'game_statistics.csv', index=False)
data.head(500).to_csv(WRITE_FOLDER_PATH+'snippets/game_statistics_snippet.csv', index=False)

941009
